# TerraFusion Property Condition Model Training v2

This notebook demonstrates how to train an improved version of the property condition model using feedback data from users.

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from datetime import datetime
import csv
from sklearn.model_selection import train_test_split

## 1. Load Feedback Data

Load the feedback data from the CSV file and analyze differences between AI scores and user scores.

In [ ]:
# Path to the feedback CSV file
FEEDBACK_CSV = '../data/condition_feedback.csv'

# Load feedback data if it exists
if os.path.exists(FEEDBACK_CSV):
    df_feedback = pd.read_csv(FEEDBACK_CSV)
    print(f"Loaded {len(df_feedback)} feedback records")
    
    # Calculate agreement statistics
    df_feedback['agreement'] = df_feedback['ai_score'] == df_feedback['user_score']
    df_feedback['difference'] = abs(df_feedback['ai_score'] - df_feedback['user_score'])
    
    agreement_rate = df_feedback['agreement'].mean()
    avg_difference = df_feedback['difference'].mean()
    
    print(f"Agreement rate: {agreement_rate:.2%}")
    print(f"Average score difference: {avg_difference:.2f}")
    
    # Identify misclassified examples (where AI and user scores differ)
    misclassified = df_feedback[df_feedback['ai_score'] != df_feedback['user_score']]
    print(f"Found {len(misclassified)} misclassified examples")
    
    # Show distribution of differences
    plt.figure(figsize=(10, 6))
    plt.hist(df_feedback['difference'], bins=10)
    plt.title('Distribution of Score Differences')
    plt.xlabel('Absolute Difference')
    plt.ylabel('Frequency')
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print(f"Feedback file not found at {FEEDBACK_CSV}")
    print("Run the application and collect user feedback first")

## 2. Prepare Dataset with Feedback-Enhanced Labeling

Create a custom dataset that includes feedback corrections.

In [ ]:
class PropertyConditionDataset(Dataset):
    """Dataset for property condition images with user feedback integration"""
    
    def __init__(self, feedback_df, uploads_dir='../uploads', transform=None):
        """
        Args:
            feedback_df: DataFrame with feedback data
            uploads_dir: Directory containing the uploaded images
            transform: Optional transform to be applied to images
        """
        self.feedback_df = feedback_df
        self.uploads_dir = uploads_dir
        self.transform = transform
        
        # Filter to keep only entries with existing image files
        valid_entries = []
        for idx, row in feedback_df.iterrows():
            img_path = os.path.join(uploads_dir, row['filename'])
            if os.path.exists(img_path):
                valid_entries.append(idx)
                
        self.feedback_df = feedback_df.iloc[valid_entries].reset_index(drop=True)
        print(f"Dataset contains {len(self.feedback_df)} valid entries with images")
    
    def __len__(self):
        return len(self.feedback_df)
    
    def __getitem__(self, idx):
        # Get image path
        img_path = os.path.join(self.uploads_dir, self.feedback_df.iloc[idx]['filename'])
        
        # Load and convert image
        image = Image.open(img_path).convert('RGB')
        
        # Apply transformations if specified
        if self.transform:
            image = self.transform(image)
        
        # Get user score as ground truth (preferred over AI score)
        # Normalize to range 0-4 for classification (from 1-5)
        score = self.feedback_df.iloc[idx]['user_score'] - 1
        
        # Return image and target score
        return image, score

## 3. Define Model Architecture

Use MobileNetV2 with pretrained weights as the base model

In [ ]:
def create_model():
    # Load pre-trained MobileNetV2 model
    model = models.mobilenet_v2(pretrained=True)
    
    # Modify the classifier to output 5 classes (condition levels 1-5)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 5)
    
    return model

## 4. Training Loop with Misclassification Prioritization

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=10, device='cpu'):
    # Initialize tracking variables
    best_model_wts = model.state_dict()
    best_acc = 0.0
    train_loss_history = []
    val_loss_history = []
    train_acc_history = []
    val_acc_history = []
    
    # Start training loop
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0
            
            # Iterate over data batches
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Zero the parameter gradients
                optimizer.zero_grad()
                
                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())
                    
                    # Backward + optimize only in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # Save history
            if phase == 'train':
                train_loss_history.append(epoch_loss)
                train_acc_history.append(epoch_acc.item())
            else:
                val_loss_history.append(epoch_loss)
                val_acc_history.append(epoch_acc.item())
            
            # Save best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict().copy()
                
        print()
    
    # Print training results
    print(f'Best val Acc: {best_acc:.4f}')
    
    # Load best model weights
    model.load_state_dict(best_model_wts)
    
    # Return model and history
    return model, {
        'train_loss': train_loss_history,
        'val_loss': val_loss_history,
        'train_acc': train_acc_history,
        'val_acc': val_acc_history
    }

## 5. Main Training Workflow with Misclassified Prioritization

In [ ]:
# Check if feedback data exists and proceed with training
if os.path.exists(FEEDBACK_CSV) and len(df_feedback) > 10:  # Ensure enough samples
    # Define transformations
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    
    # Priority sampling for misclassified examples
    # Oversample misclassified examples to focus training on them
    if len(misclassified) > 0:
        print("Applying priority sampling for misclassified examples...")
        # Duplicate misclassified samples to increase their weight
        augmented_df = pd.concat([df_feedback, misclassified, misclassified])
        print(f"Augmented dataset size: {len(augmented_df)} entries")
    else:
        augmented_df = df_feedback
    
    # Split data into train and validation sets
    train_df, val_df = train_test_split(augmented_df, test_size=0.2, random_state=42)
    
    # Create datasets
    train_dataset = PropertyConditionDataset(train_df, transform=data_transforms['train'])
    val_dataset = PropertyConditionDataset(val_df, transform=data_transforms['val'])
    
    # Create dataloaders
    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4),
        'val': DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=4)
    }
    
    # Set device for training
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    # Create the model
    model = create_model()
    model = model.to(device)
    
    # Try to load existing model weights for fine-tuning
    MODEL_PATH = '../models/condition_model.pth'
    if os.path.exists(MODEL_PATH):
        print(f"Loading existing model from {MODEL_PATH} for fine-tuning")
        model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    
    # Set up loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # Train the model
    print("\nStarting model training...")
    trained_model, history = train_model(model, dataloaders, criterion, optimizer,
                                         num_epochs=15, device=device)
    
    # Save the model
    MODEL_DIR = '../models'
    if not os.path.exists(MODEL_DIR):
        os.makedirs(MODEL_DIR)
    
    # Save with version number and timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_v2_path = f"{MODEL_DIR}/condition_model_v2_{timestamp}.pth"
    torch.save(trained_model.state_dict(), model_v2_path)
    
    # Also save as the default model
    torch.save(trained_model.state_dict(), MODEL_PATH)
    
    print(f"\nModel saved to {model_v2_path}")
    print(f"Updated default model at {MODEL_PATH}")
    
    # Plot training results
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train')
    plt.plot(history['val_loss'], label='Validation')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train')
    plt.plot(history['val_acc'], label='Validation')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # Log to model training history
    TRAINING_LOG = f"{MODEL_DIR}/training_history.csv"
    log_exists = os.path.exists(TRAINING_LOG)
    
    with open(TRAINING_LOG, 'a') as f:
        writer = csv.writer(f)
        if not log_exists:
            writer.writerow(['timestamp', 'model_version', 'training_samples', 'misclassified_samples', 'val_accuracy'])
        
        writer.writerow([
            timestamp,
            'v2',
            len(train_dataset),
            len(misclassified),
            best_acc.item()
        ])
    
    print(f"Training log updated at {TRAINING_LOG}")
    
else:
    print("Insufficient feedback data for training.")
    print("Please collect more user feedback before training the v2 model.")

## 6. Model Evaluation and Error Analysis

In [ ]:
# Analyze model performance on feedback data
def evaluate_model_on_feedback(model, feedback_df, uploads_dir='../uploads', device='cpu'):
    model.eval()
    
    # Define transform for evaluation
    eval_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    
    results = []
    
    with torch.no_grad():
        for idx, row in feedback_df.iterrows():
            img_path = os.path.join(uploads_dir, row['filename'])
            if not os.path.exists(img_path):
                continue
                
            # Load and transform image
            image = Image.open(img_path).convert('RGB')
            image_tensor = eval_transform(image).unsqueeze(0).to(device)
            
            # Get model prediction
            outputs = model(image_tensor)
            _, pred_class = torch.max(outputs, 1)
            predicted_score = pred_class.item() + 1  # Convert back to 1-5 scale
            
            # Get softmax probabilities
            probabilities = torch.nn.functional.softmax(outputs, dim=1)[0]
            
            # Calculate weighted average score
            weighted_score = 0
            for i in range(5):
                weighted_score += (i + 1) * probabilities[i].item()
                
            # Store results
            results.append({
                'filename': row['filename'],
                'ai_score_old': row['ai_score'],
                'user_score': row['user_score'],
                'predicted_score': predicted_score,
                'weighted_score': weighted_score
            })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(results)
    
    if len(results_df) > 0:
        # Calculate improvement metrics
        results_df['old_error'] = abs(results_df['ai_score_old'] - results_df['user_score'])
        results_df['new_error'] = abs(results_df['weighted_score'] - results_df['user_score'])
        results_df['improvement'] = results_df['old_error'] - results_df['new_error']
        
        avg_old_error = results_df['old_error'].mean()
        avg_new_error = results_df['new_error'].mean()
        avg_improvement = results_df['improvement'].mean()
        
        # Print results
        print(f"\nModel Evaluation Results:")
        print(f"Average error with old model: {avg_old_error:.4f}")
        print(f"Average error with new model: {avg_new_error:.4f}")
        print(f"Average improvement: {avg_improvement:.4f} ({(avg_improvement/avg_old_error)*100:.1f}%)")
        
        # Plot error comparison
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.scatter(results_df['user_score'], results_df['ai_score_old'], alpha=0.7, label='Old Model')
        plt.scatter(results_df['user_score'], results_df['weighted_score'], alpha=0.7, label='New Model')
        plt.plot([1, 5], [1, 5], 'k--')  # Ideal prediction line
        plt.xlabel('User Score')
        plt.ylabel('Model Score')
        plt.title('Score Comparison')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.hist(results_df['old_error'], alpha=0.7, bins=10, label='Old Model Error')
        plt.hist(results_df['new_error'], alpha=0.7, bins=10, label='New Model Error')
        plt.xlabel('Absolute Error')
        plt.ylabel('Frequency')
        plt.title('Error Distribution')
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        plt.tight_layout()
        plt.show()
        
        return results_df
    else:
        print("No valid images found for evaluation")
        return None

In [ ]:
# Run evaluation if model was trained successfully
if 'trained_model' in locals():
    print("Evaluating trained model on feedback data...")
    evaluation_results = evaluate_model_on_feedback(trained_model, df_feedback, device=device)
else:
    print("Model training was not completed. Cannot evaluate.")

## 7. Summary and Next Steps

- The model has been retrained with user feedback to improve accuracy
- Misclassified examples were prioritized in training
- Performance metrics show the improvement from v1 to v2
- The new model is ready for deployment

### Next Steps:

1. Continue collecting user feedback to further refine the model
2. Consider adding more types of image augmentation
3. Explore more advanced architectures or ensemble methods
4. Implement feature importance visualization (SHAP values)